This interactive Jupyter Notebook will teach the basics of how to interact with a parsl cloud execution provider. 
In this case, we are using an AWS EC2 provider.

In [4]:
from parsl import *
import parsl
from ipyparallel import Client
import aws
import time

#set up executor, dataflow kernel, and provider
executor = parsl.executors.ipp.IPyParallelExecutor()
dfk = DataFlowKernel(executor)
provider = aws.EC2Provider('providerconf.json')
with open('providerconf.json', 'r') as fin:
    print (fin.read())

{
	"site": "EC2",
	"instancetype": "t2.nano",
	"nodeGranularity": 1,
	"maxNodes": 100,
	"AMIID": "ami-ae90b6cb",
	"AWSKeyName": "benglick-key",
	"logFile":"awsprovider.log"
}



In [ ]:
#Show the aws architecture. Run this cell at any time to see what's running
print(provider.show_summary())

In [ ]:
#Submit a request to the provider to increase the size of your cluster and wait for it to connect
instances = provider.scale_out(1)
time.sleep(180)

In [ ]:
#Define a parsl app.
#In this case, we will use monte carlo pi, as it is nice and simple
import random
@App('python', dfk)
def mc_pi(n):
    import random
    count = 0
    for i in range(n):
        x = random.random()
        y = random.random()
        if (x**2 + y**2) <= 1:
            count += 1
    return count/n

#For comparison, we'll also define a serial version. 
#Note the only difference is the lack of an @App decorator
def mc_pi_serial(n):
    import random
    count = 0
    for i in range(n):
        x = random.random()
        y = random.random()
        if (x**2 + y**2) <= 1:
            count += 1
    return count/n

In [ ]:
#Time the serial version
%timeit mc_pi_serial(1000000)

In [ ]:
#scale out cluster (we had 1 node before, so we should have 4 now)
provider.scale_out(3)
time.sleep(180)

In [ ]:
#time the parallel version. Make sure to scale out the cluster before you run this
%%timeit
results = [mc_pi(n) for n in [250000]*8]
pi = sum([result.result() for result in results])/2
print(pi)

In [ ]:
#Shut down cluster
provider.scale_in(99)